In [1]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', 100)

from statsmodels.tsa.stattools import grangercausalitytests

# Global variables and paths

In [2]:
company = 'plug_power'
ticker = 'PLUG'

In [3]:
path_stocks = '../../download_stock_data/' + ticker + '.csv'
path_twitter_sentiment = '../../../../data/team/NLTK Time Series/sentiment_nltk_fixed_final.csv'

# Import Data

## Import Stock Data

In [4]:
stock = pd.read_csv(path_stocks)
stock['Date'] = pd.to_datetime(stock['timestamp'])
stock.set_index('Date', inplace=True);
stock.drop(['timestamp'],axis=1, inplace=True)
stock_close_daily = pd.DataFrame(stock['close'])

## Import Twitter Data

In [5]:
twitter_sentiment = pd.read_csv(path_twitter_sentiment)
twitter_sentiment.drop('Unnamed: 0', axis=1, inplace=True)
twitter_sentiment['Date'] = pd.to_datetime(twitter_sentiment['CreatedAt'].str[:10], errors='coerce')
twitter_sentiment.set_index('Date',inplace=True);
twitter_sentiment.drop('CreatedAt', axis=1, inplace=True)

# Select Company

In [6]:
# select twitter_sentiment data based on company
twitter_sentiment_company = twitter_sentiment[twitter_sentiment['company'] == company]

# Create daily tweet count 

In [7]:
tweets_daily = twitter_sentiment_company.groupby(['Date']).count()
tweets_daily = pd.DataFrame(tweets_daily['compound'])
tweets_daily.rename(columns={'compound':'Daily Tweets'},inplace=True)

# Create Sentiment data

In [8]:
sentiment = twitter_sentiment_company[['neg','pos','compound']]

In [9]:
sentiment_daily = pd.DataFrame(sentiment['neg']).groupby(['Date']).mean()

In [10]:
sentiment_daily.sort_index(ascending=False);

In [11]:
sentiment_daily['pos'] = pd.DataFrame(sentiment['pos']).groupby(['Date']).mean()

In [12]:
sentiment_daily['compound'] = pd.DataFrame(sentiment['compound']).groupby(['Date']).mean()

In [13]:
sentiment_daily.sort_index(ascending=False)

,neg,pos,compound
Date,,,
2021-04-01,0.078500,0.309333,0.427317
2021-03-31,0.030168,0.145186,0.234212
2021-03-30,0.030894,0.141361,0.237124
2021-03-29,0.055476,0.100949,0.112794
2021-03-28,0.049895,0.105689,0.149739
...,...,...,...
2011-01-11,0.031429,0.188857,0.223414
2011-01-10,0.000000,0.065000,0.175567
2011-01-05,0.000000,0.000000,0.000000


In [14]:
# sentiment_daily = sentiment.groupby(['Date']).count()

# Merge Data

In [15]:
data = pd.merge(tweets_daily, sentiment_daily, how='left', left_on=['Date'], right_on=['Date'] )
data = pd.merge(data, stock_close_daily, how='left', left_on=['Date'], right_on=['Date'] )
data.dropna(inplace=True)
data.rename(columns={'Daily Tweets':'daily_tweets','close':'closing_price', 'compound':'compound_sentiment', 'neg':'neg_sentiment','pos':'pos_sentiment'}, inplace=True)

# Export Data 

In [16]:
# name of output csv
output = "./data_" + company + "_"  + ticker + ".csv"

In [17]:
data.to_csv(output)

In [18]:
data

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-04,12,0.046417,0.349500,0.445792,0.4584
2011-01-05,1,0.000000,0.000000,0.000000,0.4550
2011-01-10,3,0.000000,0.065000,0.175567,0.4610
2011-01-11,7,0.031429,0.188857,0.223414,0.4800
2011-01-12,3,0.000000,0.297667,0.405800,0.5202
...,...,...,...,...,...
2021-03-26,323,0.033272,0.107440,0.163563,34.0300
2021-03-29,374,0.055476,0.100949,0.112794,30.7800
2021-03-30,396,0.030894,0.141361,0.237124,34.0400


# Normalize Data

In [19]:
def normalize_daily_tweets(df):
    max_count = df['daily_tweets'].max()
    min_count = df['daily_tweets'].min()
    df['daily_tweets'] = (df['daily_tweets'] - min_count) / (max_count - min_count)
    return df


def normalize_neg_sentiment(df):
    max_count = df['neg_sentiment'].max()
    min_count = df['neg_sentiment'].min()
    df['neg_sentiment'] = (df['neg_sentiment'] - min_count) / (max_count - min_count)
    return df


def normalize_pos_sentiment(df):
    max_count = df['pos_sentiment'].max()
    min_count = df['pos_sentiment'].min()
    df['pos_sentiment'] = (df['pos_sentiment'] - min_count) / (max_count - min_count)
    return df


def normalize_compound_sentiment(df):
    max_count = df['compound_sentiment'].max()
    min_count = df['compound_sentiment'].min()
    df['compound_sentiment'] = (df['compound_sentiment'] - min_count) / (max_count - min_count)
    return df


def normalize_closing_price(df):
    max_count = df['closing_price'].max()
    min_count = df['closing_price'].min()
    df['closing_price'] = (df['closing_price'] - min_count) / (max_count - min_count)
    return df

In [20]:
data

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-04,12,0.046417,0.349500,0.445792,0.4584
2011-01-05,1,0.000000,0.000000,0.000000,0.4550
2011-01-10,3,0.000000,0.065000,0.175567,0.4610
2011-01-11,7,0.031429,0.188857,0.223414,0.4800
2011-01-12,3,0.000000,0.297667,0.405800,0.5202
...,...,...,...,...,...
2021-03-26,323,0.033272,0.107440,0.163563,34.0300
2021-03-29,374,0.055476,0.100949,0.112794,30.7800
2021-03-30,396,0.030894,0.141361,0.237124,34.0400


In [21]:
data_normalized = normalize_daily_tweets(data)

In [22]:
data_normalized = normalize_neg_sentiment(data)

In [23]:
data_normalized = normalize_pos_sentiment(data)

In [24]:
data_normalized = normalize_compound_sentiment(data)

In [25]:
data_normalized = normalize_closing_price(data)

In [26]:
data_normalized

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-04,0.005203,0.097925,0.582500,0.725334,0.004659
2011-01-05,0.000000,0.000000,0.000000,0.466198,0.004613
2011-01-10,0.000946,0.000000,0.108333,0.568254,0.004695
2011-01-11,0.002838,0.066305,0.314762,0.596067,0.004955
2011-01-12,0.000946,0.000000,0.496111,0.702087,0.005505
...,...,...,...,...,...
2021-03-26,0.152318,0.070195,0.179066,0.561276,0.464154
2021-03-29,0.176443,0.117038,0.168249,0.531764,0.419671
2021-03-30,0.186850,0.065177,0.235602,0.604037,0.464291


In [27]:
output_normalized = "./data_" + company + "_"  + ticker + "_normalized.csv"
data.to_csv(output_normalized)
data_normalized

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-04,0.005203,0.097925,0.582500,0.725334,0.004659
2011-01-05,0.000000,0.000000,0.000000,0.466198,0.004613
2011-01-10,0.000946,0.000000,0.108333,0.568254,0.004695
2011-01-11,0.002838,0.066305,0.314762,0.596067,0.004955
2011-01-12,0.000946,0.000000,0.496111,0.702087,0.005505
...,...,...,...,...,...
2021-03-26,0.152318,0.070195,0.179066,0.561276,0.464154
2021-03-29,0.176443,0.117038,0.168249,0.531764,0.419671
2021-03-30,0.186850,0.065177,0.235602,0.604037,0.464291


In [28]:
data

,daily_tweets,neg_sentiment,pos_sentiment,compound_sentiment,closing_price
Date,,,,,
2011-01-04,0.005203,0.097925,0.582500,0.725334,0.004659
2011-01-05,0.000000,0.000000,0.000000,0.466198,0.004613
2011-01-10,0.000946,0.000000,0.108333,0.568254,0.004695
2011-01-11,0.002838,0.066305,0.314762,0.596067,0.004955
2011-01-12,0.000946,0.000000,0.496111,0.702087,0.005505
...,...,...,...,...,...
2021-03-26,0.152318,0.070195,0.179066,0.561276,0.464154
2021-03-29,0.176443,0.117038,0.168249,0.531764,0.419671
2021-03-30,0.186850,0.065177,0.235602,0.604037,0.464291
